# MLFlow
Comme dit dans le README de l'accueil, lancer plusieurs fois Pycaret est intéressant pour obtenir différents modèles avec différentes configurations. Prendre des notes pour chaque démarrage peut devenir rapidement lassant et c'est facile d'oublier une des expérimentations et ce sera ELLE qu'on aurait voulu garder.

MLFlow est un des outils qui permet de garder des traces de chacune des expérimentations. Il en existe d'autres, je vous invite à chercher ces alternatives si vous trouvez que MLFlow ne répond pas à vos besoins.

## Installation
Pour cette partie, nous allons réutiliser l'environnement déjà créé pour Pycaret. 

Voici les commandes à exécuter pour activer l'environnement pour Pycaret et pour installer MLFlow:
- `conda activate demo_pycaret`
- `pip install mlflow`

In [ ]:
import os
from random import random, randint
import mlflow
from mlflow import log_metric, log_param, log_artifacts

In [ ]:
# Code pour s'assurer du cas où vous auriez déjà roulé le code de la cellule suivante.
mlflow.end_run()

In [ ]:
with mlflow.start_run():
    with mlflow.start_run(nested=True):
        log_param("param", randint(0, 100))
        # Enregistrement d'une métrique : elles peuvent être mises à jour au cours de route
        log_metric("foo", random())
        log_metric("foo", random() + 1)
        log_metric("foo", random() + 2)
        # Enregistrement d'un fichier dans le dossier "artifacts"
        if not os.path.exists("outputs"):
            os.makedirs("outputs")
        with open("outputs/test.txt", "w") as f:
            f.write("hello world!")
        log_artifacts(f"outputs")

### Visualisation des résultats
Pour visualiser les résultats, dans un terminal (vous pouvez utiliser le même Anaconda prompt que vous utilisez depuis le début ou le lancer dans un terminal de vs code), veuillez faire pointer le chemin vers où se trouve le notebook actuel. Ainsi, si vous faites afficher la liste des fichiers (avec DIR ou LS), vous devriez voir le notebook et 2 répertoires (mlruns et outputs).

Une fois que vous pointez au bon endroit, lancer la commande : `mlflow ui`. Ceci lancera le serveur de MLFlow localement. Vous pouvez accéder à l'interface via votre navigateur à l'adresse :
- [http://localhost:5000](http://localhost:5000)

Pour fermer le serveur, dans la fenêtre de commande (anaconda prompt ou vs code), faites un `ctrl + c`.

Pour tester la visualisation, n'hésitez pas à exécuter plusieurs fois la cellule précédente.

# Jonction de Pycaret et MLFlow
Une des raisons pour laquelle je vous ai montré Pycaret et non pas un autre AutoML, c'est l'inclusion de MLFlow dans Pycaret.

Le processus de mettre MLFlow avec Pycaret s'incruste dans le processus de MLOps. Il s'agit d'un paradigme d'automatisation de l'apprentissage machine. En effet, comme nous connaissons DevOps, il est normal de vouloir déployer et d'automatiser ce processus.

Le site suivant explique bien le niveau de maturité de MLOps : [Modèle de maturité MLOps](https://docs.microsoft.com/fr-ca/azure/architecture/example-scenario/mlops/mlops-maturity-model).

Ainsi, avec MLOps, nous sommes partiellement dans un niveau d'automatisation. Dans ce contexte, nous devenons en mesure de répéter la génération des modèles avec une trace des paramètres utilisés.

In [ ]:
# Sans plus tarder, commençons Pycaret avec MLFlow.
from pycaret.datasets import get_data
data = get_data('diamond')

### EDA
l'EDA (Exploration Data Analysis - Analyse Exploratoire des Données) est une étape cruciale pour le scientifique de données. Il doit avoir une idée de ce que sont ses données. Plus il les connait, mieux il sera en mesure de valider ses modèles.

En voici un petit exemple.

In [ ]:
import plotly.express as px
fig = px.scatter(x=data['Carat Weight'], y=data['Price'], 
                 facet_col = data['Cut'], opacity = 0.25, template = 'plotly_dark', trendline='ols',
                 trendline_color_override = 'red', title = 'SARAH GETS A DIAMOND - A CASE STUDY')
fig.show()

In [ ]:
fig = px.histogram(data, x=["Price"], template = 'plotly_dark', title = 'Histogram of Price')
fig.show()

In [ ]:
import numpy as np

# Création d'une copie de data
data_copy = data.copy()

# Ajout d'une colonne à data_copy
data_copy['Log_Price'] = np.log(data['Price'])

fig = px.histogram(data_copy, x=["Log_Price"], title = 'Histgram of Log Price', template = 'plotly_dark')
fig.show()

### Pycaret avec MLFow
Nous avons déjà vu dans le notebook `Pycaret.ipynb` comment fonctionnait Pycaret, dans la préparation de la cellule suivante, saurez-vous trouver la magie de MLFlow?

In [ ]:
from pycaret.regression import *
s = setup(data, target = 'Price', transform_target = True, log_experiment = True, experiment_name = 'diamond', silent = True)

In [ ]:
best = compare_models()

In [ ]:
plot_model(best, plot = 'residuals_interactive')

In [ ]:
plot_model(best, plot = 'feature')

In [ ]:
final_best = finalize_model(best)

### Visualisation
Je vous invite à retourner dans la page où se trouve MLFlow UI dans votre navigateur. Jusqu'à présent, vous n'aviez qu'un seul projet "Default", si vous appuyez sur le rafraichissement de votre navigateur, pas celui dans l'interface de MLFlow, vous verrez apparaitre un nouveau projet "diamond". Vous pouvez voir chacune des expérimentations que Pycaret a fait.

# Exercice
Maintenant que vous savez comment fonctionne MLFlow et Pycaret, c'est l'heure des exercices!

Cette fois-ci, il n'y en aura qu'un seul. Si vous refaites l'expérimentation des cellules précédentes, vous allez vous rendre compte que toutes les expérimentations seront sous la même run. Ce n'est pas très convivial... Je vous invite à trouver une façon de faire pour que chaque fois que vous aller exécuter Pycaret, qu'il soit sous une nouvelle run, mais toujours sous le projet diamond...

In [ ]:
# Code à faire à partir d'ici. Vous pouvez créer autant de cellules que vous le souhaitez. Laissez au moins la conclusion à la fin. :)


# Conclusion
Et voilà, avec ce que vous avez vu jusqu'à présent, vous avez un début de notion de scientifique de données! Toutes mes félicitations!

Ce qu'il vous resterait à explorer :
- Faire de l'ETL (Extraction, Transformation, Loading), il s'agit d'un moyen pour rendre plus facile l'extraction des données pour le modèle.
- Faire de l'EDA (Exploration des données).
- Faire le déploiement du modèle. Pycaret offre des fonctionnalités pour déployer un modèle dans AWS (Amazon), Azure (Microsoft) ou même dans GCP (Google Cloud Plateform).
- Faire le pipeline au complet de MLOps.
- Continuer d'apprendre et de vous informer! Moi aussi, je continue d'apprendre. :)